## 04 - Merge With Feature Barcode Info

In [1]:
from libraries import *
from parameters import *


In [2]:
os.getcwd()
os.chdir(projectDir)

Please change the file name conventions according to your needs and update the code below accordingly.

In [3]:
feats = {}

for d in tqdm(sorted([x for x in Path('data').iterdir() if x.is_dir() and x.name.startswith('E')])):
    sample = d.name.replace('_', '-')
    roundlow = sample.split('-')[0].lower()
    chrom = sample.split('-')[1].lower()
    
        
    feat_path = Path(f'data/{sample.replace("-", "_")}/R{roundlow[1:]}_Ch{chrom}_FeatBarc/R{roundlow[1:]}_Ch{chrom}_FeatBarc.csv')
    if feat_path.is_file():
        print(sample)
        feats[sample] = pd.read_csv(feat_path)

  0%|          | 0/46 [00:00<?, ?it/s]

E3-1
E3-10
E3-11
E3-12
E3-13
E3-14
E3-15
E3-16
E3-17
E3-18
E3-19
E3-2
E3-20
E3-21
E3-22
E3-23
E3-24
E3-3
E3-4
E3-5
E3-6
E3-7
E3-8
E3-9
E4-1
E4-10
E4-11
E4-12
E4-13
E4-14
E4-15
E4-16
E4-17
E4-19
E4-2
E4-20
E4-21
E4-22
E4-24
E4-3
E4-4
E4-5
E4-6
E4-7
E4-8
E4-9


In [4]:
feats_new_barcode = []

for sample, df in tqdm(list(feats.items())):
    df = df.set_index('CRISPR').T
    df['mixhash'] = sample
    df.index = df.index + '-1-' + df.mixhash
    
    feats_new_barcode.append(df)

  0%|          | 0/46 [00:00<?, ?it/s]

In [5]:
feats_new_barcode = pd.concat(feats_new_barcode, axis=0)
del feats_new_barcode['mixhash']


In [6]:
feats_new_barcode.shape

(1166357, 3720)

Replace all 1s with 0s, then compute the total number of crispr reads per cell.

In [7]:
feats_new_barcode = feats_new_barcode.replace(1, 0)
rowSums = feats_new_barcode.sum(axis=1)
feats_new_barcode = feats_new_barcode[rowSums > 1]
feats_new_barcode.shape

(757147, 3720)

Filter out chimeric reads with a predefined threshold (default 0.2)

In [8]:
feats_new_barcodePerc = feats_new_barcode.copy()
feats_new_barcodePerc = feats_new_barcodePerc.div(rowSums, axis=0)
m = (feats_new_barcodePerc < par_cutoff_crispr_chimeric_reads)
feats_new_barcode[m] = 0
feats_new_barcode = feats_new_barcode[feats_new_barcode.sum(axis=1) > 0]
feats_new_barcode.shape

(754450, 3720)

In [9]:
adata = sc.read(par_save_filename_1)
mDF = adata.obs.join(feats_new_barcode,how="inner")
adata = adata[mDF.index,:]
adata.obs = adata.obs.join(feats_new_barcode,how="inner")
adata.uns['feature_barcode_names'] = feats_new_barcode.columns.tolist()

In [10]:
adata.write(par_save_filename_1)